# B&B and Consistency cuts with considering optimality cut

### Parameters

In [ ]:
# Define your lower bound here:
zeta = -1807;        
folder = ['data/6045/']
example = ['NewExample1']

In [ ]:
import math
import numpy as np
import cplex  as CPX
import time

In [ ]:
def SecModelParameter (CGLP_m, n, CGLP_rhs, CGLP_Coef,Br_var):
    MultiplyVar=np.arange(Br_var+1,n).tolist()
    Colij=dict()
    k=n
    for i in MultiplyVar:
        for j in range (n):
            if i!=j:
                if (tuple([j,i]) in Colij)!=True:
                    if i<j:
                        Colij[tuple([i,j])]= k
                        k=k+1
                    else:
                        Colij[tuple([j,i])]= k
                        k=k+1
    lenColij=len(Colij)+n
    CompleteMatrix=[]
    CompleteRHS=[]
    for i in MultiplyVar:
        for t in range (CGLP_m):
            LineMatrix=np.zeros(lenColij)
            for j in range (n):
                if i==j:
                    LineMatrix[j]= CGLP_Coef[t][j]
                    LineMatrix[j]=LineMatrix[j]-CGLP_rhs[t]
                    
                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=CGLP_Coef[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=CGLP_Coef[t][j]    
            CompleteMatrix.append(LineMatrix)
            CompleteRHS.append(0)

            LineMatrix=np.zeros(lenColij)
            for j in range (n):
                if j==i:
                    LineMatrix[i]= CGLP_rhs[t]
                else:
                    LineMatrix[j]= CGLP_Coef[t][j]
                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=-1*CGLP_Coef[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=-1*CGLP_Coef[t][j]   
            CompleteMatrix.append(LineMatrix)   
            CompleteRHS.append(CGLP_rhs[t])
        
        

    for i in MultiplyVar:
        for t in range (2*n):
            LineMatrix=np.zeros(lenColij)
            LineRHS=np.zeros(1)
            for j in range (n):
                LineRHS= my_rhsUnit[t]
                if i==j:
                    LineMatrix[j]= UnitMatrix[t][j]
                    LineMatrix[j]=LineMatrix[j]-my_rhsUnit[t] 
                if i<j:

                    LineMatrix [Colij[tuple([i,j])]]=UnitMatrix[t][j]
                if i>j:

                    LineMatrix [Colij[tuple([j,i])]]=UnitMatrix[t][j] 

            if all(k == 0 for k in LineMatrix)!=True:
                CompleteMatrix.append(LineMatrix) 
                CompleteRHS.append(0)

            LineMatrix=np.zeros(lenColij)
            LineRHS=np.zeros(1)
            for j in range (n):
                LineRHS= my_rhsUnit[t]
                if j==i:
                    LineMatrix[i]= my_rhsUnit[t]
                else:
                    LineMatrix[j]=UnitMatrix[t][j]

                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=-1*UnitMatrix[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=-1*UnitMatrix[t][j]   

            if all(k == 0 for k in LineMatrix)!=True:
                CompleteMatrix.append(LineMatrix) 
                CompleteRHS.append(LineRHS)

                    
    return CompleteMatrix,CompleteRHS


In [ ]:
def BB (m,n,A, my_obj,b, lx, ux, IntVars,my_colnames,zeta):
    lp_solving=0
    branch_flag=-1
    prev_Br_var=-2
    Br_var=-1
    node_numbers=0
    Tol = 0.0000001;
    LXS = np.array([np.copy(lx)]);
    UXS = np.array([np.copy(ux)]);
    ZS = np.array([1000000]);
    Br_order=np.array([-1])
    x = [];
    prev_zeta= zeta 
    nn = 1;
    while 1==1:
        f=list(np.where(ZS <= zeta + Tol)[0])
        if len(f)>0:
            LXS=np.delete(LXS, f, 0)
            UXS=np.delete(UXS, f, 0)
            ZS=np.delete(ZS, f)
            Br_order=np.delete(Br_order,f)
            nn = nn - len(f);
        if nn==0:
            break
        lx1=LXS[0]
        ux1=UXS[0] 
        LXS=np.delete(LXS, 0, 0)
        UXS=np.delete(UXS, 0, 0)
        ZS=np.delete(ZS, 0)
        prev_Br_var=Br_var
        Br_var=Br_order[0]
        Br_order=np.delete(Br_order,0)
        nn = nn - 1;
        node_numbers=node_numbers+1
        Lp_Relaxation = CPX.Cplex() 
        Lp_Relaxation.objective.set_sense(Lp_Relaxation.objective.sense.maximize)    
        Lp_Relaxation.variables.add(obj = my_obj, lb=lx1, ub =ux1, names = my_colnames)
        Lp_Relaxation.linear_constraints.add(lin_expr = A, senses = 'L'*m, rhs = b) 
        Lp_Relaxation.parameters.simplex.display.set(0)
        Lp_Relaxation.solve()

        status= Lp_Relaxation.solution.get_status()
        if status != 3:
            xR= Lp_Relaxation.solution.get_values()
            zR= Lp_Relaxation.solution.get_objective_value()
        if status == 2:
            check_unbound=BB (m,n,A, np.zeros[n],b, lx, ux, IntVars,my_colnames)
            zR0= check_unbound[0]
            if zR0==0:
                zeta = math.inf
                break
        if (status == 3 or zR<=zeta + Tol):
            continue 
         
        Fractions=np.abs(xR - np.round(xR))
        maxf=max(Fractions)

        if maxf <= Tol:     
            x = xR;
            zeta = zR;
            continue;  

#####################################################################################################               
        VarsValue=np.copy(lx1[0:Br_var+1]) 
        if Br_var not in [-1,n-1]:
            if Br_var>branch_flag or prev_zeta!=zeta:
                branch_flag=Br_var
                if prev_zeta!=zeta:
                    prev_zeta=zeta
                    CGLP_rhs[-1]=-1*zeta
                SecModelParameter_output=SecModelParameter(CGLP_m, n, CGLP_rhs, CGLP_Coef,Br_var)
                CompleteMatrix=SecModelParameter_output[0]
                CompleteRHS=SecModelParameter_output[1]
                numrows = len(CompleteMatrix)    
                numcols = len(CompleteMatrix[0])
                my_lb=[0.0]*numrows
                my_row=[0.0]*numrows
                my_newrow=[0.0]*n
                Newval=[-1.0]*n
                my_Newlb=[-1*CPX.infinity]*(Br_var+1)+[0]*(n-Br_var-1)
                my_Newub=[1*CPX.infinity]*(Br_var+1)+[0]*(n-Br_var-1)
                my_colnames_names=[]
                my_Newcolnames=[]
                my_Constnames=[]
                Fliped_CompleteMatrix=np.array(CompleteMatrix).T.tolist()     
                sumline=[]
                for i in range(numrows):
                    my_colnames_names.append("y"+str(i))   
                for i in range (n):
                    my_Newcolnames.append("t"+str(i)) 
                for i in range (numcols):
                    my_Constnames.append("c"+str(i)) 
                    sumline.append([range(numrows),Fliped_CompleteMatrix[i]]) 

                my_SecProb = CPX.Cplex()
                my_SecProb.objective.set_sense(my_SecProb.objective.sense.maximize)
                my_SecProb.variables.add(obj = my_row,lb=my_lb, names = my_colnames_names)
                my_SecProb.variables.add(obj = my_newrow,lb=my_Newlb,ub=my_Newub ,names = my_Newcolnames)
                my_SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E"*numcols,
                                        rhs = [0]*numcols, names=my_Constnames)

                CGLP_obj=[]
                for i in range (numrows):
                    CGLP_obj.append(-1*CompleteRHS[i])
                rows=[]
                rows.append([range(numrows),CGLP_obj])
                my_SecProb.linear_constraints.add(lin_expr = rows, senses = "G", rhs = [1], names = ["p0"])  
                for i in range (n):
                    my_SecProb.linear_constraints.set_coefficients("c"+str(i), my_Newcolnames[i], -1.0)    
                    
            for i in range (n):
                if i <= Br_var:
                    my_SecProb.linear_constraints.set_coefficients("p0", my_Newcolnames[i],VarsValue[i])

            
            my_SecProb.parameters.simplex.display.set(0)
            my_SecProb.solve()
            lp_solving=lp_solving+1
            SecProbStatus=my_SecProb.solution.get_status()

            if SecProbStatus not in [3,25,11]:
                continue 
#####################################################################################################                
        indf=Br_var+1       
        nlx = np.array(np.copy(lx1));
        nlx[indf] = 1;    
        nux = np.array(np.copy(ux1));
        nux[indf] = 0;
        LXS=np.append(LXS,(lx1, nlx),axis=0)
        UXS=np.append(UXS,(nux, ux1),axis=0)
        ZS = np.append(ZS, (zR, zR));
        Br_order=np.append(Br_order,(indf,indf))
        nn = nn + 2
    return zeta,node_numbers,lp_solving  

In [ ]:
for x in folder:
    for y in example:
        main_prob = CPX.Cplex(x+y+'.lp')
        m=main_prob.linear_constraints.get_num()
        CGLP_m=m+1
        n=main_prob.variables.get_num()
        my_rhs=[]
        CGLP_rhs=[]
        my_coef=[]
        CGLP_Coef=[]
        for i in range(m):
            temp=[]
            for j in range(n):
                temp.append (main_prob.linear_constraints.get_coefficients(i,j))    
            my_coef.append([list (range(n)),temp])
            CGLP_Coef.append(temp)
        
        CGLP_Coef = np.vstack((CGLP_Coef, main_prob.objective.get_linear()))
        CGLP_Coef[-1]=np.negative(CGLP_Coef[-1])
        for i in range (m):
            my_rhs=np.append(my_rhs,main_prob.linear_constraints.get_rhs(i))
            CGLP_rhs=np.append(CGLP_rhs,main_prob.linear_constraints.get_rhs(i))
        CGLP_rhs=np.append(CGLP_rhs,zeta*-1)
        my_obj= np.array(main_prob.objective.get_linear())
        UnitMatrix=np.eye(n, dtype=int).tolist()+np.negative(np.eye(n, dtype=int)).tolist()
        my_rhsUnit=[1]*n+[0]*n 
        IntVars=np.arange(n)
        lx= [0.0]*n
        ux=[1.0]*n
        my_colnames=[]
        for i in range(n):
            my_colnames.append("x"+str(i))  

        time1=time.time()
        output=BB (m,n,my_coef,my_obj,my_rhs, lx, ux, IntVars,my_colnames,zeta)
        time2=time.time()-time1

        with open(x+y+'bb_consistency_optimality.txt', 'a') as f:
            f.write('{}:{}\n'.format('objective value',output[0]))
            f.write('{}:{}\n'.format('Number of explored node',output[1]))
            f.write('{}:{}\n'.format('Number of solved LPs',output[2]))
            f.write('{}:{}\n'.format('ElapsedTime',time2))
        f.closed